**IMPORT LIBRARIES**

In [ ]:
import tensorflow as tf
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
tf.__version__

'2.2.0'

**MOUNT GOOGLE DRIVE FILES**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**ACCESS THE DATASET**

In [ ]:
base_dir = '/content/drive/My Drive/Face Detection Project'
train_dir = os.path.join(base_dir, 'Train')
val_dir = os.path.join(base_dir,'Validation')
train_face_dir = os.path.join(train_dir, 'Face')
train_no_face_dir = os.path.join(train_dir, 'New No Face')
val_face_dir = os.path.join(val_dir, "Face")
val_no_face_dir = os.path.join(val_dir, "New No Face")

In [ ]:
print(str(len(os.listdir(train_face_dir))) + ' training examples of Faces')
print(str(len(os.listdir(train_no_face_dir))) + ' training examples of No Faces')
train_total = len(os.listdir(train_face_dir)) + len(os.listdir(train_no_face_dir))
print(str(train_total) + ' training examples')
print(str(len(os.listdir(val_face_dir))) + ' validation examples of Faces')
print(str(len(os.listdir(val_no_face_dir))) + ' validation examples of No Faces')
val_total = len(os.listdir(val_face_dir)) + len(os.listdir(val_no_face_dir))
print(str(val_total) + ' validation examples')
total = train_total + val_total
print('Total ' + str(total) + ' examples')

6278 training examples of Faces
6068 training examples of No Faces
12346 training examples
1221 validation examples of Faces
1167 validation examples of No Faces
2388 validation examples
Total 14734 examples


**DATA PREPROCESSING AND GENERATING BATCHES FOR TRAINING**

In [ ]:
ht = 420
wd = 300
batch_size = 32

def center(img):
    img = np.array(img)  
    img = cv2.resize(img, (wd, ht))
    img = img/255.0
    img[:,:,0] -= 0.5519912
    # img[:,:,0] /= 0.3125070
    img[:,:,1] -= 0.4811025
    # img[:,:,1] /= 0.2979582
    img[:,:,2] -= 0.4498843
    # img[:,:,2] /= 0.2945640
    return img

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,rotation_range=30,
                                                                brightness_range=[0.8, 1.2], preprocessing_function=center)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=center)

In [ ]:
train_gen = train_datagen.flow_from_directory(train_dir, 
                                              batch_size = batch_size, 
                                              subset = 'training',
                                              target_size = (ht, wd),
                                              class_mode = 'binary',
                                              shuffle = True)

val_gen = val_datagen.flow_from_directory(val_dir, 
                                          batch_size = batch_size, 
                                          subset = None,
                                          target_size = (ht, wd),
                                          class_mode = 'binary',
                                          shuffle = True)

Found 12346 images belonging to 2 classes.
Found 2388 images belonging to 2 classes.


In [ ]:
def my_activation_fun(x):
  # return tf.keras.backend.switch(x>0, tf.keras.backend.relu(x), 0)
  return tf.keras.backend.relu(x, max_value=10, threshold=-1)

In [ ]:
# def my_activation_fun(x):
#   return tf.cond(x > 0, lambda: tf.multiply(x, 1), lambda: tf.multiply(x, 0))
#   # return tf.multiply(x, 0.8)

**MODEL ARCHITECTURE**

In [ ]:
regularizer = tf.keras.regularizers.l2(l=0)
regularizer_1 = tf.keras.regularizers.l2(l=0.001)
init = tf.initializers.glorot_uniform()
model = tf.keras.models.Sequential([
                                    
        tf.keras.layers.Conv2D(filters = 16, kernel_size = (3, 3), strides = (1, 1), padding='same', activation=my_activation_fun, kernel_initializer=init, kernel_regularizer=regularizer, input_shape=(ht, wd, 3)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(16, (5, 5), (1, 1), padding='same', activation=my_activation_fun, kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
        
        tf.keras.layers.Conv2D(32, (3, 3), (1, 1), padding='same', activation=my_activation_fun, kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(32, (5, 5), (1, 1), padding='same', activation=my_activation_fun, kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),

        tf.keras.layers.Conv2D(64, (3, 3), (1, 1), padding='same', activation=my_activation_fun, kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, (5, 5), (1, 1), padding='same', activation=my_activation_fun, kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
    
        tf.keras.layers.Conv2D(96, (3, 3), (1, 1), padding='same', activation=my_activation_fun, kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Conv2D(96, (5, 5), (1, 1), padding='same', activation=my_activation_fun, kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
        tf.keras.layers.SpatialDropout2D(0.2),
        
        tf.keras.layers.Conv2D(128, (3, 3), (1, 1), padding='same', activation=my_activation_fun, kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Conv2D(128, (5, 5), (1, 1), padding='same', activation=my_activation_fun, kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
        tf.keras.layers.SpatialDropout2D(0.2),
        
        tf.keras.layers.Conv2D(192, (3, 3), (1, 1), padding='same', activation=my_activation_fun, kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(192, (5, 5), (1, 1), padding='same', activation=my_activation_fun, kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
        tf.keras.layers.SpatialDropout2D(0.2),

        
        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(256, activation = 'relu', kernel_initializer=init, kernel_regularizer= regularizer_1),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 420, 300, 16)      448       
_________________________________________________________________
batch_normalization (BatchNo (None, 420, 300, 16)      64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 420, 300, 16)      6416      
_________________________________________________________________
batch_normalization_1 (Batch (None, 420, 300, 16)      64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 210, 150, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 210, 150, 32)      4640      
_________________________________________________________________
batch_normalization_2 (Batch (None, 210, 150, 32)      1

**COMPILE MODEL**

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='binary_crossentropy', metrics = ['accuracy'])

**TRAIN MODEL**

In [ ]:
def scheduler(epoch):
  if epoch <= 14:
    return 0.001
  elif 14 < epoch <= 30:
    return 0.0001
  else:
    return 0.00005

reduce_lr_1 = tf.keras.callbacks.LearningRateScheduler(scheduler)

#callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
path = '/content/drive/My Drive/Face Detection Project/Models/model_custom_activation_function_2.h5'
save_model = tf.keras.callbacks.ModelCheckpoint(path,monitor='val_loss', save_best_only=True, load_weights_on_restart=True)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
reduce_lr_2 = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                 factor=0.2, 
                                                 patience=2, 
                                                 verbose=0, 
                                                 mode='auto',
                                                 min_delta=0.0001, 
                                                 cooldown=1,
                                                 min_lr=0)

history = model.fit_generator(train_gen,
                              validation_data=val_gen,
                              steps_per_epoch=386,
                              shuffle=True, 
                              epochs=30,
                              validation_steps=75,
                              callbacks=[save_model, reduce_lr_2])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
  3/386 [..............................] - ETA: 1:34:52 - loss: 1.3374 - accuracy: 0.5938

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


171/386 [============>.................] - ETA: 1:41:14 - loss: 1.1756 - accuracy: 0.6497

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


386/386 [==============================] - 13719s 36s/step - loss: 1.0112 - accuracy: 0.6873 - val_loss: 0.9758 - val_accuracy: 0.6185 - lr: 0.0010
Epoch 2/30
386/386 [==============================] - 466s 1s/step - loss: 0.6667 - accuracy: 0.7603 - val_loss: 0.7234 - val_accuracy: 0.7240 - lr: 0.0010
Epoch 3/30
386/386 [==============================] - 450s 1s/step - loss: 0.5297 - accuracy: 0.7956 - val_loss: 0.5401 - val_accuracy: 0.7747 - lr: 0.0010
Epoch 4/30
386/386 [==============================] - 436s 1s/step - loss: 0.4458 - accuracy: 0.8297 - val_loss: 0.5200 - val_accuracy: 0.8011 - lr: 0.0010
Epoch 5/30
386/386 [==============================] - 424s 1s/step - loss: 0.4135 - accuracy: 0.8415 - val_loss: 0.6641 - val_accuracy: 0.7286 - lr: 0.0010
Epoch 6/30
386/386 [==============================] - 428s 1s/step - loss: 0.3938 - accuracy: 0.8532 - val_loss: 0.4673 - val_accuracy: 0.8065 - lr: 0.0010
Epoch 7/30
386/386 [==============================] - 429s 1s/step - los

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss') 
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

**SAVE MODEL (IF NO CALLBACK)**

In [ ]:
model.save('/content/drive/My Drive/Face Detection Project/Models/model.h5')

**TEST YOUR MODEL HERE**

In [ ]:
test_dir_1 = os.path.join(base_dir,'Test')
test_dir = os.path.join(test_dir_1, 'Test_Mask')
test_dir_file = os.path.join(test_dir,'Mask')
f = 1
test_dir_file_name = os.listdir(test_dir_file)
ht = 150
wd = 180
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                              samplewise_center=True,
                                                              samplewise_std_normalization=True)
test_gen = test_datagen.flow_from_directory(
                test_dir,
                target_size=(ht, wd),
                batch_size= 1,
                class_mode='binary',
                shuffle=False)

a = new_model.predict_generator(test_gen)
#a = np.around(a, 4)

#plt.figure(figsize=(30, 30))
j = 1
cnf = 0
cf = 0
for i in a:
    #plt.subplot(10,5,j)
    #img = mpimg.imread('C:/Face Detection/Dataset/Test_Mask/Mask/'+test_dir_file_name[j-1])
    #plt.imshow(img)
    if i > 0.5:
        cnf += 1
        #plt.title('No Face '+str(1-i), fontdict={'color':'g'})
    else:
        #plt.title('Face '+str(1-i), fontdict={'color':'r'})
        cf += 1
        
    j += 1
#plt.savefig('C:/Face Detection/Dataset/Test/Face/prediction.jpg')
print('no. of faces = ' + str(cf))
print('no. of no faces = ' + str(cnf))
if f == 0:
    accuracy = cf/len(a)*100
else:
    accuracy = cnf/len(a)*100
print(accuracy)

**OTHER MODEL ARCHITECTURES**

In [ ]:
regularizer = tf.keras.regularizers.l2(l=0.001)
init = tf.initializers.he_uniform()
model = tf.keras.models.Sequential([
                                    
        tf.keras.layers.Conv2D(filters = 32, kernel_size = (3, 3), strides = (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer, input_shape=(ht, wd, 3)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        
        tf.keras.layers.Conv2D(32, (5, 5), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
        
        tf.keras.layers.Conv2D(64, (3, 3), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
        tf.keras.layers.Dropout(0.1),
        
        tf.keras.layers.Conv2D(128, (3, 3), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
        tf.keras.layers.Dropout(0.1),

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(256, activation = 'relu', kernel_regularizer= regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.summary()

In [ ]:
regularizer = tf.keras.regularizers.l2(l=0.1)
init = tf.initializers.he_uniform()
model = tf.keras.models.Sequential([
                                    
        tf.keras.layers.Conv2D(filters = 32, kernel_size = (3, 3), strides = (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer, input_shape=(ht, wd, 3)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        
        tf.keras.layers.Conv2D(64, (5, 5), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
        
        tf.keras.layers.Conv2D(96, (3, 3), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
        tf.keras.layers.Dropout(0.1),
        
        tf.keras.layers.Conv2D(128, (3, 3), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
        tf.keras.layers.Dropout(0.1),

        tf.keras.layers.Conv2D(256, (3, 3), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
        tf.keras.layers.Dropout(0.1),

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(256, activation = 'relu', kernel_regularizer= regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.summary()

In [ ]:
regularizer = tf.keras.regularizers.l2(l=0)
regularizer_1 = tf.keras.regularizers.l2(l=0)
init = tf.initializers.he_uniform()
model = tf.keras.models.Sequential([
                                    
        tf.keras.layers.Conv2D(filters = 32, kernel_size = (3, 3), strides = (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer, input_shape=(ht, wd, 3)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.DepthwiseConv2D(3, (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
        
        tf.keras.layers.Conv2D(64, (3, 3), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, (3, 3), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.DepthwiseConv2D(3, (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
    
        tf.keras.layers.Conv2D(96, (3, 3), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Conv2D(96, (3, 3), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.DepthwiseConv2D(3, (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
        tf.keras.layers.SpatialDropout2D(0.2),
        
        tf.keras.layers.Conv2D(128, (3, 3), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Conv2D(128, (3, 3), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.DepthwiseConv2D(3, (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
        tf.keras.layers.SpatialDropout2D(0.25),
        
        tf.keras.layers.Conv2D(192, (3, 3), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(192, (3, 3), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.DepthwiseConv2D(3, (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
        tf.keras.layers.SpatialDropout2D(0.3),

        tf.keras.layers.Conv2D(256, (3, 3), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(256, (3, 3), (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.DepthwiseConv2D(3, (1, 1), padding='same', activation='relu', kernel_initializer=init, kernel_regularizer=regularizer),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), (2, 2)),
        tf.keras.layers.SpatialDropout2D(0.3),
        
        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(256, activation = 'relu', kernel_initializer=init, kernel_regularizer= regularizer_1),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.summary()